In [2]:
# Load and organize data according to directions at 
# https://web.archive.org/web/20201012220940/https://www.drivendata.co/blog/genetic-attribution-benchmark/

In [3]:
import pandas as pd
from itertools import permutations
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import make_pipeline

In [4]:
train_values = pd.read_csv(r'/Users/elizabeth/Documents/GitHub/Build-Week-2/data/train_values.csv', index_col='sequence_id')
train_labels = pd.read_csv(r'/Users/elizabeth/Documents/GitHub/Build-Week-2/data/train_labels.csv', index_col='sequence_id')
test_values = pd.read_csv(r'/Users/elizabeth/Documents/GitHub/Build-Week-2/data/test_values_.csv', index_col='sequence_id')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/elizabeth/Documents/GitHub/Build-Week-2/data/train_values.csv'

In [ ]:
train_values.head()

In [ ]:
sequence_lengths = train_values.sequence.apply(len)

In [ ]:
train_values.iloc[:, 1:].apply(pd.value_counts)

In [ ]:
sorted_binary_features = train_values.iloc[:, 1:].mean().sort_values()

In [ ]:
train_labels.head()

In [ ]:
lab_ids = pd.DataFrame(train_labels.idxmax(axis=1), columns=['lab_id'])

In [ ]:
lab_ids.head()

In [ ]:
lab_ids.shape

In [ ]:
lab_ids['lab_id'].value_counts().describe()

In [ ]:
bases = set(''.join(train_values.sequence.values))

In [ ]:
bases

In [ ]:
subsequences = [''.join(permutation) for permutation in permutations(bases, r=4)]

In [ ]:
print(f"Number of subsequences: {len(subsequences)}")

In [ ]:
def get_ngram_features(data, subsequences):
    """Generates counts for each subsequence.

    Args:
        data (DataFrame): The data you want to create features from. Must include a "sequence" column.
        subsequences (list): A list of subsequences to count.

    Returns:
        DataFrame: A DataFrame with one column for each subsequence.
    """
    features = pd.DataFrame(index=data.index)
    for subseq in subsequences:
        features[subseq] = data.sequence.str.count(subseq)
    return features

In [ ]:
ngram_features = get_ngram_features(train_values, subsequences)
ngram_features.head()

In [ ]:
ngram_features.shape

In [ ]:
all_features = ngram_features.join(train_values.drop('sequence', axis=1))

In [ ]:
all_features.shape

In [ ]:
# Create custom scorer according to the directions at
# https://web.archive.org/web/20201012220940/https://www.drivendata.co/blog/genetic-attribution-benchmark/

In [ ]:
def top10_accuracy_scorer(estimator, X, y):
    """A custom scorer that evaluates a model on whether the correct label is in 
    the top 10 most probable predictions.

    Args:
        estimator (sklearn estimator): The sklearn model that should be evaluated.
        X (numpy array): The validation data.
        y (numpy array): The ground truth labels.

    Returns:
        float: Accuracy of the model as defined by the proportion of predictions
               in which the correct label was in the top 10. Higher is better.
    """
    # predict the probabilities across all possible labels for rows in our training set
    probas = estimator.predict_proba(X)
    
    # get the indices for top 10 predictions for each row; these are the last ten in each row
    # Note: We use argpartition, which is O(n), vs argsort, which uses the quicksort algorithm 
    # by default and is O(n^2) in the worst case. We can do this because we only need the top ten
    # partitioned, not in sorted order.
    # Documentation: https://numpy.org/doc/1.18/reference/generated/numpy.argpartition.html
    top10_idx = np.argpartition(probas, -10, axis=1)[:, -10:]
    
    # index into the classes list using the top ten indices to get the class names
    top10_preds = estimator.classes_[top10_idx]

    # check if y-true is in top 10 for each set of predictions
    mask = top10_preds == y.reshape((y.size, 1))
    
    # take the mean
    top_10_accuracy = mask.any(axis=1).mean()
 
    return top_10_accuracy

In [ ]:
X=all_features
y=lab_ids.values.ravel()

In [ ]:
# 2.3.1.1 Choose your target. Which column in your tabular dataset will you predict?
# I'm predicting lab_ids.

#2.3.1.2 Is your problem regression or classification?
# Classification.

#2.3.1.3 How is your target distributed?
# My target is distributed unevenly.
df_y.value_counts().describe()

In [ ]:
#2.3.1.4 Classification: How many classes? Are the classes imbalanced?
# My target is very imbalanced.  There are 1314 possible lab IDs, but the top 3 labs account for 20% of the sequences.

#2.3.1.5 Choose your evaluation metric(s).
# My evaluation metric is the custom scorer, top_ten_accuracy.

#2.3.1.6 Choose which observations you will use to train, validate, and test your model.
# The test X is test_values.
# I will use cross-validation, so I don't need to split my train and validation data.